In [ ]:
### If save data:
save_data = False

if save_data:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd drive/My Drive/

In [ ]:
!pip install cupy-cuda100
import cupy as cp

import numpy as np
import matplotlib.pyplot as plt

!pip install -q git+https://github.com/Pehlevan-Group/kernel-generalization
from kernel_generalization import kernel_simulation as ker_sim
from kernel_generalization import kernel_spectrum as ker_spec
from kernel_generalization import kernel_regression as ker_reg
from kernel_generalization.utils import gegenbauer

from datetime import datetime
import os

def plot_exp_shift(P_vals, All_errs, std_errs, all_mc, std_MC, sol, errs_tot, alpha, shift, n_idx, directory, label, save_data = False):
    
    All_errs = All_errs[:,:,:,n_idx]
    std_errs = std_errs[:,:,:,n_idx]
    all_mc = all_mc[:,:,n_idx]
    std_MC = std_MC[:,:,n_idx]
    
    sol = sol[:,:,:,n_idx]
    errs_tot = errs_tot[:,:,n_idx]
    alpha = alpha[n_idx]
    
    plt.rcParams.update({'font.size': 12})
    fig, axs = plt.subplots(1, 2)
    
    fig.set_size_inches(7, 4)
    
    kplot = [1, 2, 3,4]
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    
    scale = [1e+9**i for i in range(len(kplot))]
    
    for i in range(len(kplot)):
        for j in range(len(shift)):
            error_exp = All_errs[j, :, kplot[i]]/All_errs[j, 0, kplot[i]]/scale[i]
            std_exp = std_errs[j, :, kplot[i]]/All_errs[j, 0, kplot[i]]/scale[i]
            
            if j == 0:
                axs[0].errorbar(P_vals, error_exp, std_exp, fmt = 'o' , color=colors[i], label='k=%d' % kplot[i])
            else:
                axs[0].errorbar(P_vals, error_exp, std_exp, fmt = 'o' , color=colors[i])
            
            error_theory = sol[j, :, kplot[i]]/sol[j, 0, kplot[i]]/scale[i]
            axs[0].plot(P_vals, error_theory, '--', color=colors[i])
            
            axs[0].set_xscale('log')
            axs[0].set_yscale('log')
            axs[0].set_xlabel(r'P')
            axs[0].set_ylabel(r'$E_k$')
            axs[0].legend()
    
    for j in range(len(shift)):
        error_exp = all_mc[j,:]#/all_mc[j,0]
        std_exp = std_MC[j,:]#/all_mc[j,0]
        alpha_s = alpha+shift[j]
        axs[1].errorbar(P_vals, error_exp, std_exp, fmt = 'o', color=colors[j],label='$\\alpha^*$=%f' % alpha_s)
        
        error_theory = errs_tot[j,:]#/errs_tot[j,0]
        axs[1].plot(P_vals, error_theory, '--', color=colors[j])
        
        axs[1].set_xscale('log')
        axs[1].set_yscale('log')
        axs[1].set_xlabel(r'P')
        axs[1].set_ylabel(r'$E_g$')
        axs[1].legend()
    
    plt.subplots_adjust(bottom=0.02, right=0.8, top=0.7, left=-0.25)
    fig.set_figheight(7)
    fig.set_figwidth(12)
    fig.set
    
    plt.xlim([np.amin(P_vals) + 5, np.amax(P_vals)])
    
    if save_data: plt.savefig(directory +label+ 'error_curve_'+str(n_idx)+'.pdf', bbox_inches = 'tight')
    
    plt.show()
    
def noise_s(alpha):
    trig_left = (-np.cos(np.arctan(8*np.sqrt(alpha)/(-1+9*alpha*(2+3*alpha)))/3+np.pi/3)+np.sqrt(3)*np.sin(np.arctan(8*np.sqrt(alpha)/(-1+9*alpha*(2+3*alpha)))/3+np.pi/3))
    trig_right = (-np.cos(np.arctan(8*np.sqrt(alpha)/(-1+9*alpha*(2+3*alpha)))/3)+np.sqrt(3)*np.sin(np.arctan(8*np.sqrt(alpha)/(-1+9*alpha*(2+3*alpha)))/3))
    return 3*alpha*(3*alpha + 2) + 3*(alpha*np.sqrt(1+alpha)*np.sqrt(1+9*alpha))*(trig_left*(alpha<(np.sqrt(12)-np.sqrt(9))/9)+trig_right*(alpha>=(np.sqrt(12)-np.sqrt(9))/9))

def Noise(spectrum, lambda_bar, lamb, cut, shift):
    kmax = len(spectrum)
    alpha_s = lamb+np.array([np.sum(lambda_bar[i+1:])/lambda_bar[i+1] for i in range(kmax-2)])
    
    noise = np.zeros((len(shift),cut))
    for i in range(cut):
        for j in range(len(shift)):
            if alpha_s[i] >= 2:
                alpha = (alpha_s[i]-1)*(1+shift[j])
                noise[j,i] = spectrum[i+1]**2*(2*alpha+1)*degens[i+1]
            else:
                alpha = (alpha_s[i]-1)*(1+shift[j])
                noise[j,i] = spectrum[i+1]**2*noise_s(alpha)*degens[i+1]
                
    return noise, alpha_s


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/NatComm_Codes/RBF/'
/content/drive/My Drive/Colab Notebooks/NatComm_Codes/RBF



/content/drive/My Drive/Colab Notebooks/NatComm_Codes/RBF/kernel_regression.py:13: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 1d, A))
  return np.dot(X,Y)


In [ ]:
# Parameters of the experiment

P_vals = np.logspace(0.25, 4.1, num=30).astype('int')
dim = 20
kmax = 100
lamb = 0
num_repeats_before_100 = 1000
num_repeats_after_100 = 30
P_teach = 50
P_test = 1000

# Obtain the spectrum and degenerecies
s = 8
degens = np.array([gegenbauer.degeneracy_kernel(dim, k) for k in range(kmax)])

spec_type = ['exp','power']
spec = 1

if spec_type[spec] == 'exp':
    spectrum, lambda_bar = ker_spec.exp_spectrum(s, kmax, degens)
    spectrum[0] =1
    lambda_bar[0] = 0
    alpha = np.array([np.sum(lambda_bar[i+1:])/lambda_bar[i+1] for i in range(kmax-2)])
    noise = lambda_bar[1:5]*alpha[0:4]/degens[1:5]
elif spec_type[spec] == 'power':
    spectrum, lambda_bar = ker_spec.power_spectrum(s, kmax, degens)
    spectrum[0] = 0
    lambda_bar[0] = 0
    cut = 4
    shift = np.array([-0.8,0,2,5])
    noise, alpha = Noise(spectrum,lambda_bar,lamb,cut,shift)
    
# Calculate theory

Sol = np.zeros((len(shift), len(P_vals), kmax, cut))
Errs_tot = np.zeros((len(shift), len(P_vals), cut))
Cum_gen_errs = np.zeros((len(shift), len(P_vals), kmax, cut))

for i in range(len(shift)):
    Sol[i], Errs_tot[i] = ker_sim.simulate_gen_error(P_vals, spectrum, degens, noise[i], lamb=lamb)
                                                                                    
                                                                                    
# Save initial files

params = {'Spectrum': spec_type[spec], 'decay(s)': s, 'dim': dim, 'kmax': kmax, 'noise': noise, 'shift': shift, 
          'lamb': lamb, 'teach_samp': P_teach, 'test_samp': P_test, 'num_repeat<100': num_repeats_before_100, 'num_repeat>100': num_repeats_after_100}


if save_data:
  timestamp = datetime.now().strftime("%m-%d_%H-%M")
  comment = '_synth'
  directory = timestamp + '_dim_'+ str(dim)+ '_' + spec_type[spec]+'_' + str(s) + comment + '/'
  os.mkdir(directory)
  filename_theory = directory + timestamp + '_error_theory.npz'
  np.savez(filename_theory, P_vals, Sol, Errs_tot, Cum_gen_errs, alpha, params)

# Perform the experiment

All_errs = np.zeros((len(shift), len(P_vals), kmax, cut))
all_mc = np.zeros((len(shift), len(P_vals), cut))
std_errs = np.zeros((len(shift), len(P_vals), kmax, cut))
std_MC = np.zeros((len(shift), len(P_vals), cut))

for i in range(len(P_vals)):
    print('\n ####### remaining: ' + str(len(P_vals) - i) + ' #######', end = '\r')
    
    for j in range(len(shift)):
        P = P_vals[i]
        if P < 100:
            num_repeats = num_repeats_before_100
        else:
            num_repeats = num_repeats_after_100
            
        All_errs[j,i, :, :], all_mc[j,i, :], std_errs[j,i, :, :], std_MC[j,i, :] = ker_reg.generalization(P,P_teach, P_test,
                                                                                        spectrum, degens,
                                                                                        dim, kmax,
                                                                                        num_repeats,
                                                                                        lamb=lamb,
                                                                                        noise_var=noise[j])
                                                                                        
    ##### Saving data #####
    if P > 10:
        timestamp = datetime.now().strftime("%d_%H-%M")
        
        if save_data:
          label = timestamp + '_' + str(P) + '_'
          filename_exp = directory + label + 'error_exp.npz'
          np.savez(filename_exp, All_errs, all_mc, std_errs, std_MC)
        
        for n_idx in range(noise.shape[1]):
            plot_exp_shift(P_vals[:i], All_errs[:,:i], std_errs[:,:i], all_mc[:,:i], std_MC[:,:i], 
                           Sol[:,:i], Errs_tot[:,:i], alpha, shift, n_idx, directory, label)                                                                               


timestamp = datetime.now().strftime("%d_%H-%M")
label = timestamp + '_' + str(P_vals[i]) + '_'

if save_data:
  filename_exp = directory + label + 'error_exp.npz'
  np.savez(filename_exp, All_errs, all_mc, std_errs, std_MC)

for n_idx in range(noise.shape[1]):
    plot_exp_shift(P_vals[:i], All_errs[:,:i], std_errs[:,:i], all_mc[:,:i], std_MC[:,:i], Sol[:,:i], Errs_tot[:,:i], alpha, shift, n_idx, directory, label)
